In [ ]:
!pip install ortools
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random
from scipy.stats import spearmanr


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
def leitura(arquivo):

    ARQ = pd.read_excel(arquivo, sheet_name='Qtde')
    DMU = int(ARQ.at[0,'Valores'])
    IN = int(ARQ.at[1,'Valores'])
    OUT = int(ARQ.at[2,'Valores'])
    ANOS = int(ARQ.at[3,'Valores'])
    Nomes = []
    for k in range(DMU):
        nome = ARQ.at[k,'Nomes']
        Nomes.append(nome)
    Anos = []
    for k in range(ANOS):
        ano = ARQ.at[k,'Anos']
        Anos.append(ano)

    ARQ = pd.read_excel(arquivo, sheet_name='NomesIO')
    NomesI = []
    NomesO = []
    for i in range(IN):
        n = ARQ.at[i,'Inputs']
        NomesI.append(n)
    for i in range(OUT):
        n = ARQ.at[i,'Outputs']
        NomesO.append(n)

    parametros_anuais = dict()
    for ano in Anos:
        x = []
        y = []
        for i in range(IN):
            ARQ = pd.read_excel(arquivo, sheet_name='Input'+str(i+1))
            nomecoluna = ano
            xI = []
            for k in range(DMU):
                xIK = ARQ.at[k, nomecoluna]
                xI.append(xIK)
            x.append(xI)
        for j in range(OUT):
            ARQ = pd.read_excel(arquivo, sheet_name='Output'+str(j+1))
            nomecoluna = ano
            yJ = []
            for k in range(DMU):
                yJK = ARQ.at[k, nomecoluna]
                yJ.append(yJK)
            y.append(yJ)
        parametros_anuais[ano] = (x,y)

    return DMU, IN, OUT, ANOS, Anos, Nomes, parametros_anuais, NomesI, NomesO

def inverte(y):
    yy = []
    for lista in y:
        lista2 = []
        for elem in lista:
            elem2 = 1/elem
            lista2.append(elem2)
        yy.append(lista2)
    return yy

def subtrai(y):
    maior = [0]*len(y)
    menor = [float('inf')]*len(y)
    for j in range(len(y)):
        for elem in y[j]:
            if elem > maior[j]:
                maior[j] = elem
            if elem < menor[j]:
                menor[j] = elem
    yy = []
    for j in range(len(y)):
        lista2 = []
        K = 2.5*maior[j]
        for elem in y[j]:
            elem2 = K - elem
            lista2.append(elem2)
        yy.append(lista2)
    return yy

def bcc_multiplicadores_output(ano, dmuO, Nomes, DMU, IN, OUT, x, y, ineficientes, eficientes, resultado):

    v = [0]*IN
    u = [0]*OUT

    pl = pywraplp.Solver('BCC - Output', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

    for i in range(IN):
        v[i] = pl.NumVar(0, pl.infinity(), 'v'+str(i))  # min, max, nome

    for j in range(OUT):
        u[j] = pl.NumVar(0, pl.infinity(), 'u'+str(j))  # min, max, nome

    ve = pl.NumVar(-pl.infinity(), pl.infinity(), 've')

    pl.Add( pl.Sum(u[j]*y[j][dmuO] for j in range(OUT)) == 1 )

    restD = [0]*DMU
    for k in range(DMU):
        restD[k] = pl.Constraint(-pl.infinity(), 0)
        restD[k].SetCoefficient( ve, -1)
        for j in range(OUT):
            restD[k].SetCoefficient( u[j], float(y[j][k]))
        for i in range(IN):
            restD[k].SetCoefficient( v[i], -float(x[i][k]))

    pl.Minimize( pl.Sum(v[i]*float(x[i][dmuO]) for i in range(IN)) + ve )

    status = pl.Solve()
    eff = 100*(1/pl.Objective().Value())
    resultado[ano, Nomes[dmuO]] = eff

    if eff < 99.9:
        multiplicadorOut = pl.Objective().Value()
        referencia = []
        for k, constraint in enumerate(restD):
            dual = abs(constraint.dual_value())
            if dual > 0:
                referencia.append(Nomes[k])
        ineficientes[ano, Nomes[dmuO]] = round(eff,6)
    else:
        vf = [round(e.solution_value(),2) for e in v]
        uf = [round(e.solution_value(),2) for e in u]
        eficientes[ano, Nomes[dmuO]] = 100

    return



In [ ]:
DMU, IN, OUT, ANOS, Anos, Nomes, parametros_anuais, NomesI, NomesO = leitura("dados_banco_mundial_Resultados_predicao.xlsx")

In [ ]:

ineficientes = dict()
eficientes = dict()
resultado = dict()
for ano in Anos:
    (x, y) = parametros_anuais[ano]
    for o in range(DMU):
        bcc_multiplicadores_output(ano, o, Nomes, DMU, IN, OUT, x, y, ineficientes, eficientes, resultado)

print("\nEficientes: (ano, UF): (eff, pesos) ")
for chave, valor in eficientes.items():
    print(f"{chave}: {valor}")

print("\nIneficientes: (ano, UF): (eff, multiplicadorOut, referência)")
for chave, valor in ineficientes.items():
    print(f"{chave}: {valor}")

In [ ]:
mapping = {
    'Argentina':       'ARG',
    'Austria':         'AUT',
    'Finland':         'FIN',
    'Hungary':         'HUN',
    'Iceland':         'ISL',
    'Ireland':         'IRL',
    'Israel':          'ISR',
    'Italy':           'ITA',
    'Mexico':          'MEX',
    'Norway':          'NOR',
    'Peru':            'PER',
    'Slovak Republic': 'SVK',
    'Spain':           'ESP',
    'Sweden':          'SWE',
    'Switzerland':     'CHE',

}

dados = {
    (metric, mapping.get(country, country)): value
    for (metric, country), value in dados.items()
}

In [ ]:
paises = sorted(set(p for _, p in dados.keys()))
reais = [dados.get(('passo1_real_2016', pais), None) for pais in paises]
preditos = [dados.get(('passo1_predito_2016', pais), None) for pais in paises]
atuais = [dados.get(('valor_atual_2015', pais), None) for pais in paises]


rho, p_value = spearmanr(reais, atuais)
print(f"Spearman’s rho = {rho:.4f}, p-value = {p_value:.4g}")



fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=reais+[reais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Observed Future (2016)',
    line=dict(color='royalblue', width=4, dash='solid')
))



fig.add_trace(go.Scatterpolar(
    r=atuais+[atuais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Current (2015)',
    line=dict(color='crimson', width=2, dash='dash')
))


fig.update_layout(

    template='plotly_white',
    height=700,
    width=700,
    margin=dict(l=40, r=40, t=40, b=40),

    legend=dict(
        x=1,
        y=0.95,
        xanchor='left',
        yanchor='top',
        font=dict(size=20, color='black')
    ),
    polar=dict(
        angularaxis=dict(
            tickfont=dict(size=20)
        ),
        radialaxis=dict(
            visible=True,
            range=[80, 100],
            tickfont=dict(size=20)
        )
    ),
    showlegend=True
)


fig.show()






diferencas = [(r - a)/r for r, a in zip(reais, atuais)]



fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=diferencas + [diferencas[0]],
    theta=paises   + [paises[0]],
    fill='none',
    name='% Gap',
    line=dict(color='crimson', width=5)
))

fig.add_trace(go.Scatterpolar(
    r=[0] * (len(paises) + 1),
    theta=paises   + [paises[0]],
    mode='lines',
    showlegend=False,
    line=dict(color='black', width=4, dash='dot')
))

fig.update_layout(
    title_text='',
    template='plotly_white',
    height=700,
    width=700,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-0.1, 0.025],
            gridcolor='lightgray',
            tickfont=dict(
                size=22,
                color='black'
            ),
            tickvals=[-0.1,  0, 0.025],
            ticktext=['-0.1', '0', '0.025'],
            tickangle=0
        ),
        angularaxis=dict(
            tickfont=dict(size=20, color='black')
        )
    ),
    legend=dict(
        x=1, y=0.95,
        xanchor='left', yanchor='top',
        font=dict(size=28, color='black')
    ),
    showlegend=True,
    margin=dict(t=0)
)

fig.show()


## Observed Future (2016) VS Predicted (2016)



In [ ]:
print(reais+[reais[0]])
print(preditos+[preditos[0]])
rho, p_value = spearmanr(reais, preditos)
print(f"Spearman’s rho = {rho:.4f}, p-value = {p_value:.4g}")

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=reais+[reais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Observed Future (2016)',
    line=dict(color='royalblue', width=4, dash='solid')
))


fig.add_trace(go.Scatterpolar(
    r=preditos+[preditos[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Predicted (2016)',
    line=dict(color='Forestgreen', width=2, dash='dash')
))




fig.update_layout(
    font=dict(
        family="Arial",
        size=24,
        color="black"
    ),
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[80, 100],
            tickfont=dict(size=24),
        ),
        angularaxis=dict(
            tickfont=dict(size=24),
        )
    ),
    legend=dict(
        font=dict(size=24),
    ),
    hoverlabel=dict(
        font_size=24
    ),
    height=700,
    width=700,
    template='plotly_white',
    showlegend=True,
    margin=dict(t=50, b=50, l=50, r=50)
)

fig.show()








diferencas = [(r - p)/r for r, p in zip(reais, preditos)]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=diferencas + [diferencas[0]],
    theta=paises + [paises[0]],
    fill='none',
    name='Relative Error',
    line=dict(color='crimson', width=3)
))

fig.add_trace(go.Scatterpolar(
    r=[0] * (len(paises) + 1),
    theta=paises + [paises[0]],
    mode='lines',
    showlegend=False,
    line=dict(color='black',
              width=4,
              dash='dot')
))

fig.update_layout(
    title_text='',
    template='plotly_white',
    height=700,
    width=700,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-0.1, 0.025],
            gridcolor='lightgray',
            tickfont=dict(
                size=22,
                color='black'
            ),
                        tickvals=[-0.1,  0, 0.025],
            ticktext=['-0.1', '0', '0.025'],
            tickangle=0
        ),
        angularaxis=dict(
            tickfont=dict(
                size=17,
                color='black'
            )
        )
    ),
    legend=dict(
        x=1, y=0.95,
        xanchor='left', yanchor='top',
        font=dict(size=20, color='black')
    ),
    showlegend=True,
    margin=dict(t=0)
)


fig.show()


In [ ]:
paises = sorted(set(p for _, p in dados.keys()))
reais = [dados.get(('passo3_real_2016', pais), None) for pais in paises]
preditos = [dados.get(('passo3_predito_2016', pais), None) for pais in paises]
atuais = [dados.get(('valor_atual_2013', pais), None) for pais in paises]

rho, p_value = spearmanr(reais, atuais)
print(f"Spearman’s rho = {rho:.4f}, p-value = {p_value:.4g}")

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=reais+[reais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Observed Future (2016)',
    line=dict(color='royalblue', width=4, dash='solid')
))



fig.add_trace(go.Scatterpolar(
    r=atuais+[atuais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Current (2013)',
    line=dict(color='crimson', width=2, dash='dash')
))


fig.update_layout(
    template='plotly_white',
    height=700,
    width=700,
    legend=dict(
        x=1,
        y=0.95,
        xanchor='left',
        yanchor='top',
        font=dict(size=20, color='black')
    ),
    polar=dict(
        angularaxis=dict(
            tickfont=dict(size=20)
        ),
        radialaxis=dict(
            visible=True,
            range=[80, 100],
            tickfont=dict(size=20)
        )
    ),
    showlegend=True
)


fig.show()





diferencas = [(r - a)/r for r, a in zip(reais, atuais)]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=diferencas + [diferencas[0]],
    theta=paises + [paises[0]],
    fill='none',
    name='% Gap',
    line=dict(color='crimson', width=3)
))

fig.add_trace(go.Scatterpolar(
    r=[0] * (len(paises) + 1),
    theta=paises + [paises[0]],
    mode='lines',
    showlegend=False,
    line=dict(color='black',
              width=4,
              dash='dot')
))

fig.update_layout(
    title_text='',
    template='plotly_white',
    height=700,
    width=700,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-0.1, 0.025],
            gridcolor='lightgray',
            tickfont=dict(
                size=22,
                color='black'
            ),
                        tickvals=[-0.1,  0, 0.025],
            ticktext=['-0.1', '0', '0.025'],
            tickangle=0
        ),
        angularaxis=dict(
            tickfont=dict(
                size=17,
                color='black'
            )
        )
    ),
    legend=dict(
        x=1, y=0.95,
        xanchor='left', yanchor='top',
        font=dict(size=20, color='black')
    ),
    showlegend=True,
    margin=dict(t=0)
)


fig.show()

## Gap Future observed (2016) VS Predicted (2016)

In [ ]:

rho, p_value = spearmanr(reais, preditos)
print(f"Spearman’s rho = {rho:.4f}, p-value = {p_value:.4g}")

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=reais+[reais[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Observed Future (2016)',
    line=dict(color='royalblue', width=4, dash='solid')
))


fig.add_trace(go.Scatterpolar(
    r=preditos+[preditos[0]],
    theta=paises+[paises[0]],
    fill='none',
    name='Predicted (2016)',
    line=dict(color='Forestgreen', width=2, dash='dash')
))



fig.update_layout(
    font=dict(
        family="Arial",
        size=24,
        color="black"
    ),
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[80, 100],
            tickfont=dict(size=24),
        ),
        angularaxis=dict(
            tickfont=dict(size=24),
        )
    ),
    legend=dict(
        font=dict(size=24),
    ),
    hoverlabel=dict(
        font_size=24
    ),
    height=700,
    width=700,
    template='plotly_white',
    showlegend=True,
    margin=dict(t=50, b=50, l=50, r=50)
)

fig.show()








diferencas = [(r - p)/r for r, p in zip(reais, preditos)]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=diferencas + [diferencas[0]],
    theta=paises + [paises[0]],
    fill='none',
    name='Relative Error',
    line=dict(color='crimson', width=3)
))

fig.add_trace(go.Scatterpolar(
    r=[0] * (len(paises) + 1),
    theta=paises + [paises[0]],
    mode='lines',
    showlegend=False,
    line=dict(color='black',
              width=4,
              dash='dot')
))

fig.update_layout(
    title_text='',
    template='plotly_white',
    height=700,
    width=700,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-0.1, 0.025],
            gridcolor='lightgray',
            tickfont=dict(
                size=22,
                color='black'
            ),
                        tickvals=[-0.1,  0, 0.025],
            ticktext=['-0.1', '0', '0.025'],
            tickangle=0
        ),
        angularaxis=dict(
            tickfont=dict(
                size=17,
                color='black'
            )
        )
    ),
    legend=dict(
        x=1, y=0.95,
        xanchor='left', yanchor='top',
        font=dict(size=20, color='black')
    ),
    showlegend=True,
    margin=dict(t=0)
)


fig.show()
